In [15]:
# Equipo 3
# Importación de librerias
import mysql.connector
import pandas as pd
import numpy as np

import make_pipeline from sklearn.pipeline 
import StandardScaler, OneHotEncoder from sklearn.preprocessing 
import SimpleImputer from sklearn.impute 
import make_column_transformer from sklearn.compose 
import train_test_split from sklearn.model_selection 

# Conexión al Data Warehouse 
mydb = mysql.connector.connect(
  host = "topline.dwh.suite.ghp.mx",
  user = "tec_Aa1625364",
  password = "7JFaz0BaayQY2xls",
  database = "dwh"
)

1a. Selecciona tus variables dependientes (Y) e independientes (X)

1b. Verifica bien de qué tipo son las variables.

    Region [categorica] independiente
    Ciudad [categorica] independiente
    Estado [categorica] independiente

    Chain [categorica] independiente
    Brand [categorica] independiente

    Occupancy [numérica] independiente/dependiente
    Revenue [numérica] dependiente

1c. Verifica si necesitan algún proceso de estandarización y balanceo.
    StandardScaler()
    RobustScaler()    
    MinMaxScaler()

In [ ]:
num_attribs = ["occupancy", "revenue"]
cat_attribs = ["chain", "brand", "region", "city", "state"]

# Pipeline para las columnas numéricas
basic_num_pipeline = make_pipeline(SimpleImputer(strategy='median'), StandardScaler())

# Pipeline para las columnas categoricas
basic_cat_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(handle_unknown='ignore'))

# Pipeline de preprocesamiento integrado
basic_preprocessing = make_column_transformer((basic_num_pipeline, num_attribs), (basic_cat_pipeline, cat_attribs))

In [ ]:
# Preprocesamiento de datos
descriptors = ['occupancy', 'revenue'] 

labels = ['???']

X_train, X_test, y_train, y_test = train_test_split(housing[descriptors], housing[labels], train_size=0.8)

In [ ]:
# Transformamos los datos
housing_train_prep_array = basic_preprocessing.fit_transform(X_train)

2a. A nivel anual:
    2ai. Explora los datos para buscar patrones de estacionalidad
    2aii. Explora los datos para buscar tendencias
    2aiii. Explora los datos para buscar si hay presencia de ruido

2b. A nivel mensual:
    2bi. Explora los datos para buscar patrones de estacionalidad
    2bii. Explora los datos para buscar tendencias
    2biii. Explora los datos para buscar si hay presencia de ruido

2c. A nivel semanal:
    2ci. Explora los datos para buscar patrones de estacionalidad
    2cii. Explora los datos para buscar tendencias
    2ciii. Explora los datos para buscar si hay presencia de ruido

2d. A nivel diario:
    2di. Explora los datos para buscar patrones de estacionalidad
    2dii. Explora los datos para buscar tendencias
    2diii. Explora los datos para buscar si hay presencia de ruido